In [140]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import LinearNDInterpolator, RegularGridInterpolator
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from tqdm import tqdm
from scipy.optimize import curve_fit
import pickle

In [141]:
def transform_df(df, max_wt, rescale=True):
    avgs = df.unstack().reset_index(level=[0, 1])
    avgs.columns = ["Phe", "BH4", "Enzyme Activity"]
    if rescale:
        avgs["Enzyme Activity"] = (avgs["Enzyme Activity"] / max_wt) * 100
    avgs[avgs < 0] = 0

    x = avgs.Phe.to_numpy()
    y = avgs.BH4.to_numpy()
    z = np.float64(avgs["Enzyme Activity"].to_numpy())
    z = np.round(z, decimals=2)
    return x, y, z


def plot_landscape(
    x,
    y,
    z,
    name=" ",
    method="regular_grid",
    show=True,
    save=False,
    save_dir = "",
    nbins=1000,
    rescale=True,
    max_val_scale=None,
    legend=[True, True, True, True],
):
    x_dense = np.linspace(0, x.max(), nbins)
    y_dense = np.linspace(0, y.max(), nbins)
    B1, B2 = np.meshgrid(x_dense, y_dense, indexing="xy")
    dense_points = np.stack([B1.ravel(), B2.ravel()], -1)  # shape (N, 2) in 2d

    try:
        if method == "linear_ndi":
            scattered_points = np.stack(
                [x.ravel(), y.ravel()], -1
            )  # shape (N, 2) in 2d
            smooth_z = LinearNDInterpolator(
                scattered_points,
                z.ravel(),
                rescale=True,
                fill_value=0.0,
            )
            z_smoothed = smooth_z(dense_points).reshape(B1.shape)
            z_smoothed = ndimage.gaussian_filter(z_smoothed, sigma=10)
            z_smoothed[z_smoothed < 0] = 0

        if method == "regular_grid":
            Z = z.reshape(len(np.unique(x)), len(np.unique(y)))
            rgi = RegularGridInterpolator(
                (np.unique(x), np.unique(y)),
                Z,
                method="linear",
                bounds_error=False,
                fill_value=0.0,
            )
            Z_rgi = rgi(np.array([B1.flatten(), B2.flatten()]).T).reshape(B1.shape)
            z_smoothed = ndimage.gaussian_filter(Z_rgi, sigma=7)
            z_smoothed[z_smoothed < 0] = 0
            # z_smoothed[z_smoothed > 100] = 100

        elif method not in ["linear_ndi", "regular_grid"]:
            raise NotImplementedError(
                "Smoothing method not implemented. Choose between 'regular_grid' or 'linear_ndi'."
            )

    except NotImplementedError as e:
        print("Error: ", e)
        sys.exit()

    fig, ax = plt.subplots(figsize=(6, 5))
    if rescale:
        im = ax.pcolormesh(
            B1,
            B2,
            z_smoothed,
            cmap="Spectral_r",
            norm=colors.PowerNorm(vmin=0, vmax=100, gamma=0.6),
        )
    else:
        if max_val_scale is None:
            max_val_scale = np.max(z_smoothed)

        im = ax.pcolormesh(
            B1,
            B2,
            z_smoothed,
            cmap="Spectral_r",
            norm=colors.PowerNorm(vmin=0, vmax=max_val_scale, gamma=0.6),
        )
    ax.set_xlim([np.min(x_dense), np.max(x_dense)])
    ax.set_ylim([np.min(y_dense), np.max(y_dense)])
    ax.set_xlabel("Phe [uM]")
    ax.set_ylabel("BH4 [uM]")
    ax.set_title(f"{name}")

    CS = ax.contour(
        B1, B2, z_smoothed, 5, colors=("lightgrey"), linewidths=1, origin="lower"
    )
    ax.clabel(CS, fmt="%.0f", colors="lightgrey", fontsize=9)
    cbar = fig.colorbar(im, shrink=0.7, ax=ax)
    if rescale:
        cbar.set_label("Enzyme Activity [%]")
    else:
        cbar.set_label("Enzyme Activity")

    df = pd.DataFrame(z_smoothed, columns=x_dense, index=y_dense)

    mx = np.around(df.max().max(), decimals=2)
    bh4 = df.stack().idxmax()[0]
    phe = df.stack().idxmax()[1]

    df_tmp = df.loc[bh4, :]
    phe_min = df_tmp[df_tmp >= mx * 0.5].index.min()
    phe_max = df_tmp[df_tmp >= mx * 0.5].index.max()
    df_tmp2 = df.loc[:, phe]
    bh4_min = df_tmp2[df_tmp2 >= mx * 0.5].index.min()
    bh4_max = df_tmp2[df_tmp2 >= mx * 0.5].index.max()

    output_vals = {
        "Max": mx,
        "Phe": phe,
        "BH4": bh4,
        "50% max BH4 min": bh4_min,
        "50% max BH4 max": bh4_max,
        "50% max Phe min": phe_min,
        "50% max Phe max": phe_max,
    }

    ax.plot(
        phe,
        bh4,
        marker="x",
        c="m",
        markersize=8,
        markeredgecolor="m",
        markeredgewidth=3,
    )

    C50 = ax.contour(
        B1,
        B2,
        z_smoothed,
        levels=[mx * 0.5],
        colors=("magenta",),
        linewidths=1,
        origin="lower",
    )
    ax.clabel(C50, fmt="%.0f", colors="magenta", fontsize=9)

    info_box, max_val, peak_coords, fifty_coords = legend
    if info_box:
        pro = dict(boxstyle="round", facecolor="w", alpha=0.5)
        textstr = ""
        if max_val:
            textstr += f"Max: {mx:.0f}\n"
        if peak_coords:
            textstr += f"Phe: {phe:.0f}\nBH4: {bh4:.0f}\n"
        if fifty_coords:
            textstr += f"50% max BH4:\n{bh4_min:.0f}-{bh4_max:.0f}\n50% max Phe:\n{phe_min:.0f}-{phe_max:.0f}"
        ax.text(
            0.97,
            0.97,
            textstr,
            transform=im.axes.transAxes,
            fontsize=7,
            verticalalignment="top",
            bbox=pro,
            ha="right",
            color="k",
        )
    # im.clim(0, 100)
    plt.tight_layout()

    if save:
        plt.savefig(os.path.join(save_dir, f"landscape_{name}.png"))
    if show:
        plt.show()
    plt.close()

    return output_vals


def gaussian_2d(xy, a, mx, my, sx, sy):
    x, y = xy
    z = a * np.exp( - 0.5 * ( ((x-mx)**2 / (sx**2)) + ((y-my)**2 / (sy**2)) ) )
    return z


eps = 0.000001

In [142]:
cols_1 = ["BLANK", 25, 50, 75, 100, 125, 150, 200, 500, 750, 1250, 2500]
cols_1 = [str(col) for col in cols_1]
cols_2 = ["0.0","24.1","48.2","72.3","96.4","120.5","144.6","192.8","482.1","723.1","1205.2","2410.3"]
cols_2 = [str(col) for col in cols_2]
cols_final = [0, 25, 50, 75, 100, 125, 150, 200, 500, 750, 1250, 2500]

rows = [0, 10, 25, 50, 75, 100, 150, 250]
rows = [int(row) for row in rows]

dict_data = {}
exp = "Data/experiments_all/"
wbs = [file for file in os.listdir(exp) if ".xlsm" in file]

for wb in tqdm(wbs):
    fp = os.path.join(exp, wb)
    xl = pd.ExcelFile(fp)
    sheets = [sheet for sheet in xl.sheet_names if "-av" in sheet]
    exp_name = wb.split("_")[1].split(".")[0]
    dict_data[exp_name] = {}
    for sheet in sheets:
        df = pd.read_excel(fp, sheet_name=sheet, skiprows=4)
        mask = np.column_stack([df[col].astype(str).str.contains("BLANK", na=False) for col in df])
        if mask.any():
            cols = cols_1
        else:
            cols = cols_2
        df.columns = [str(col) for col in df.columns]
        df = df[cols]
        df = df.dropna()
        exps = [df.iloc[0:8], df.iloc[9:17], df.iloc[18:26], df.iloc[27:35]]
        for i in range(len(exps)):
            exps[i].index = rows  
            exps[i].columns = cols_final
        dict_data[exp_name][sheet.split("-av")[0]] = exps

100%|████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:22<00:00,  1.12it/s]


In [143]:
dict_data.keys()

dict_keys(['REP9', 'REP5', 'REP18', 'REP4', 'REP14', 'REP8', 'RUS5', 'REP3', 'REP12', 'REP2', 'EXP14', 'REP11', 'EXP8', 'REP10', 'UK1', 'REP17', 'RUS6', 'EXP3', 'EXP12', 'REP7', 'REP6', 'EXP13', 'EXP2', 'RUS7', 'REP16'])

In [144]:
dict_data['REP9'].keys()

dict_keys(['WT', 'A403V-A403V', 'A403V-R408W', 'I65T-R408W', 'R158Q-R261Q', 'R158Q-Y414C', 'R261Q-A403V', 'I65T-I65T'])

In [145]:
dict_data['REP9']['WT'][0]

,0,25,50,75,100,125,150,200,500,750,1250,2500
0,-41.748793,-2.396218,-12.395689,0.227104,-40.975510,9.757237,34.277525,47.043641,39.103387,8.076277,64.058514,27.177879
10,-30.903269,214.463620,1063.364100,2639.395897,3155.406608,3183.896789,3260.268547,3614.605484,3663.131702,3583.826470,3027.307793,2161.972510
25,32.368987,242.395282,1459.159167,3680.108717,5117.773715,6776.170193,6952.875449,7724.788205,7020.096710,6779.041914,5472.439101,3205.735114
50,-39.339185,291.325200,1571.175090,4167.982668,5849.031318,5749.380089,8072.745198,9921.218589,8994.994142,8220.522614,5838.158926,3535.985029
75,-1.339877,304.615626,1651.218462,4396.871595,6705.762996,7524.631680,7500.794330,10415.480250,9734.821773,8108.809325,6089.261114,3414.041094
100,-68.339783,324.854415,1653.286303,4014.323144,6268.261715,7428.535962,8475.062467,10963.721140,10056.060242,8706.300858,5925.495217,3704.636769
150,11.235571,357.123787,1490.078941,4202.457134,6415.444792,7749.330909,8946.980958,10408.443872,10901.566905,8300.153522,6039.397530,3145.826817
250,0.637031,289.261939,1568.012399,4008.260143,5995.944801,7239.194038,8085.075959,9811.893045,8088.902340,6394.427660,5177.712678,2473.586046


In [150]:
save_plot = True
save_dir = "Data/Landscapes_all"

feature = pd.DataFrame(columns=['genotype', 'experiment', 'Max', 'Max_x', 'Max_y', 's_x', 's_y', 'mse'])

for exp in tqdm(dict_data.keys()):
    variants = dict_data[exp]
    
    WT_av = variants['WT'][3]
    max_wt = np.around(WT_av.max().max(), decimals=2) # maximum value of WT to be used for rescaling

    for var in variants.keys():
        data = variants[var][3]
        name = exp + "_" + var
        
        # Get x, y, z
        x, y, z = transform_df(data, max_wt, rescale=True)
        if save_plot:
            plot_landscape(x, y, z, name = name, 
                           show = False, save = True, save_dir = save_dir)
        x, y, z = np.log(x + eps), np.log(y + eps), z

        # Curve fitting
        initial_guess = (1.0, 5, 5, 2, 2)
        bounds = ([0, eps, eps, 0.5, 0.5],                      # Lower bounds
                  [120, np.log(1500), np.log(150), 1000, 1000])  # Upper bounds
        popt, pcov = curve_fit(gaussian_2d, (x, y), z, p0=initial_guess, bounds=bounds)
        
        
        # Calculate mase
        a, mx, my, sx, sy = tuple(popt)
        z_hat = gaussian_2d((x, y), a, mx, my, sx, sy)
        mse = np.mean((z/z.max() - z_hat/z_hat.max())**2)
        if save_plot:
            plot_landscape(np.exp(x), np.exp(y), z_hat, name = name + "_model",
                          show = False, save = True, save_dir = save_dir)

        
        # Save features
        feature.loc[len(feature)] = [var, exp, a, mx, my, sx, sy, mse]

100%|████████████████████████████████████████████████████████████████████████████████████| 25/25 [08:17<00:00, 19.91s/it]


In [152]:
feature

,genotype,experiment,Max,Max_x,Max_y,s_x,s_y,mse
0,WT,REP9,104.388421,5.856702,4.600635,1.183108,1.753430,0.003206
1,A403V-A403V,REP9,23.962256,0.586823,4.190179,3.543135,2.387371,0.004995
2,A403V-R408W,REP9,7.456938,0.000001,4.181969,4.308900,2.119291,0.005917
3,I65T-R408W,REP9,5.110190,5.723829,4.307583,1.428692,2.109644,0.054861
4,R158Q-R261Q,REP9,8.142140,6.479639,4.573848,1.201338,1.891705,0.001844
...,...,...,...,...,...,...,...,...
155,R158Q-Y414C,REP16,5.043721,5.398559,4.684254,1.656303,2.603221,0.012378
156,P211T-R408Q,REP16,22.682881,5.005239,4.793704,1.466450,2.431487,0.003777
157,P225T-P281L,REP16,1.067659,7.289579,5.010635,2.630463,1.862802,0.040841
158,P281L-A300S,REP16,5.789020,5.940680,4.681701,1.508285,2.201709,0.005209


In [153]:
feature.to_csv("Data/extracted_features_v2.csv", index=False)